In [1]:
import pandas as pd

In [2]:
df = pd.read_json(path_or_buf='gg2013.json')
df.head(3)

,id,text,timestamp_ms,user
0,290620657987887104,JLo's dress! #eredcarpet #GoldenGlobes,2013-01-14 00:45:38,"{'screen_name': 'Dozaaa_xo', 'id': 557374298}"
1,290620657887219713,What's making Sofia Vergara's boobs stay like ...,2013-01-14 00:45:38,"{'screen_name': 'theAmberShow', 'id': 14648726}"
2,290620657828524032,RT @FabSugar: Kerry Washington is EVERYTHING. ...,2013-01-14 00:45:38,"{'screen_name': 'SweetyPW', 'id': 35498686}"


In [3]:
len(df)

174643

In [4]:
data = df['text']
data[:5]

0               JLo's dress! #eredcarpet #GoldenGlobes
1    What's making Sofia Vergara's boobs stay like ...
2    RT @FabSugar: Kerry Washington is EVERYTHING. ...
3       Anne Hathaway has got me living. #GoldenGlobes
4    Jennifer Lopez's lace dress? Thoughts? #Golden...
Name: text, dtype: object

In [5]:
import re

In [38]:
# awards identified from awards.ipynb
awards = ['best actress in a motion picture comedy or musical',
          'best television series actor drama',
          'best actress in a motion picture musical or comedy',
          'best supporting actor in a motion picture',
          'best supporting actor actress',
          'best supporting actress in a motion picture',
          'best actor in a motion picture comedy or musical',
          'best motion picture comedy or musical',
          'best motion picture drama',
          'best television series comedy or musical',
          'best supporting actor in a television series',
          'best motion picture television',
          'best actor in a motion picture drama',
          'best actor in a motion picture drama performance',
          'best actress in a television series drama',
          'best television series drama',
          'best supporting actress in a series',
          'best actor in a television series comedy or musical',
          'best actor in a comedy or musical',
          'best animated feature film',
          'best actor comedy or musical',
          'best actress in a motion picture drama']

In [125]:
def reduce(line):
    # remove words "or", "in" "a"
    return re.sub(r'\bin\b|\ba\b', ' ', line).split()

# test
reduce('best actress in a motion picture comedy or musical')

['best', 'actress', 'motion', 'picture', 'comedy', 'or', 'musical']

In [153]:
def generate_reduced_awards_and_award_num_keywords_map(awards):
    reduced_awards = []
    award_num_keywords_map = {}
    for i, award in enumerate([reduce(award) for award in awards]):
        if 'or' in award:
            award.remove('or')
            reduced_awards.append(award)
        award_num_keywords_map[i] = len(award) - 1 # do not count "best"
    return reduced_awards, award_num_keywords_map

In [154]:
reduced_awards, award_num_keywords_map = generate_reduced_awards_and_award_num_keywords_map(awards)

In [155]:
import pprint
pprint.pprint(awards_reduced)
sorted(award_num_keywords_map.items(), key=lambda pair: pair[0])

[['best', 'actress', 'motion', 'picture', 'comedy', 'musical'],
 ['best', 'television', 'series', 'actor', 'drama'],
 ['best', 'actress', 'motion', 'picture', 'musical', 'comedy'],
 ['best', 'supporting', 'actor', 'motion', 'picture'],
 ['best', 'supporting', 'actor', 'actress'],
 ['best', 'supporting', 'actress', 'motion', 'picture'],
 ['best', 'actor', 'motion', 'picture', 'comedy', 'musical'],
 ['best', 'motion', 'picture', 'comedy', 'musical'],
 ['best', 'motion', 'picture', 'drama'],
 ['best', 'television', 'series', 'comedy', 'musical'],
 ['best', 'supporting', 'actor', 'television', 'series'],
 ['best', 'motion', 'picture', 'television'],
 ['best', 'actor', 'motion', 'picture', 'drama'],
 ['best', 'actor', 'motion', 'picture', 'drama', 'performance'],
 ['best', 'actress', 'television', 'series', 'drama'],
 ['best', 'television', 'series', 'drama'],
 ['best', 'supporting', 'actress', 'series'],
 ['best', 'actor', 'television', 'series', 'comedy', 'musical'],
 ['best', 'actor', 'c

[(0, 5),
 (1, 4),
 (2, 5),
 (3, 4),
 (4, 3),
 (5, 4),
 (6, 5),
 (7, 4),
 (8, 3),
 (9, 4),
 (10, 4),
 (11, 3),
 (12, 4),
 (13, 5),
 (14, 4),
 (15, 3),
 (16, 3),
 (17, 5),
 (18, 3),
 (19, 3),
 (20, 3),
 (21, 4)]

In [25]:
def cleanse(line):
    # replace everything to ' ' except alphanumeric character, whitespace, apostrophe, hashtag
    return re.sub(r'[^\w\s\'#]', ' ', line)

# test
cleanse("Hugh Jackman is so awesome!!! #goldenglobes")

'Hugh Jackman is so awesome    #goldenglobes'

In [60]:
# test
set(awards_reduced[0])

{'actress', 'best', 'comedy', 'motion', 'musical', 'picture'}

In [26]:
import spacy
# python3 -m spacy download en
nlp = spacy.load('en')

In [27]:
def identify_entities(text, verbose=False):
    ents = list(nlp(text).ents)
    if verbose:
        person_tags = {}
        tags = {}
        for ent in ents:    
            entity = ' '.join(t.orth_ for t in ent).strip()
            if entity not in tags:
                tags[entity]=[ent.label_]
                if ent.label_ == 'PERSON':
                    person_tags[entity]=[ent.label_]
        return tags
    else:
        return [' '.join(t.orth_ for t in ent).strip() for ent in ents]

In [162]:
def find_award_winner(award_index, verbose=False):
    # add word bounary '\b' to prevent grabbing examples like "showing" and "wonder"
    pattern = re.compile(r'(\bwin)|(\bwon\b)', re.IGNORECASE)
    entity_dict = {}
    
    print('Predicting for:', awards[award_index])
    
    num = 0
    for tweet in data:
        line = cleanse(tweet)
        
        award = awards_reduced[award_index]
        num_keywords_to_match = award_num_keywords_map[award_index]
        
        # test for 'best actress in a motion picture comedy or musical'
#         if len(set(award).intersection(set(line.split()))) > 5:
        
        if len(set(award).intersection(set(line.split()))) > num_keywords_to_match:
            
            match = re.findall(pattern, line.lower())
            if match:
                entities = identify_entities(line, False)

                if verbose:
                    # print the first 5 occurrences
                    if num < 5:
                        print(match)
                        print(line)
                        print(entities)
                        print()

                for entity in entities:
                    if len(entity) > 1:
                        if entity not in entity_dict:
                            entity_dict[entity] = 1
                        else:
                            entity_dict[entity] += 1
                num += 1
            
    print('num of matches:', num)
    return entity_dict

In [163]:
%%time
# test
entity_dict = find_award_winner(0, verbose=True)

Predicting for: best actress in a motion picture comedy or musical
[('win', '')]
RT  ABC  Jennifer Lawrence wins best actress  motion picture comedy or musical  for  Silver Lining Playbook  http   t co r51EfFPg #GoldenGlobes
['RT', 'ABC', 'Jennifer Lawrence', '', '', 'Silver Lining Playbook', '', 'GoldenGlobes']

[('win', '')]
RT  ABC  Jennifer Lawrence wins best actress  motion picture comedy or musical  for  Silver Lining Playbook  http   t co r51EfFPg #GoldenGlobes
['RT', 'ABC', 'Jennifer Lawrence', '', '', 'Silver Lining Playbook', '', 'GoldenGlobes']

[('win', '')]
RT  ABC  Jennifer Lawrence wins best actress  motion picture comedy or musical  for  Silver Lining Playbook  http   t co r51EfFPg #GoldenGlobes
['RT', 'ABC', 'Jennifer Lawrence', '', '', 'Silver Lining Playbook', '', 'GoldenGlobes']

[('win', '')]
RT  ABC  Jennifer Lawrence wins best actress  motion picture comedy or musical  for  Silver Lining Playbook  http   t co r51EfFPg #GoldenGlobes
['RT', 'ABC', 'Jennifer Lawrenc

In [164]:
from fuzzywuzzy import fuzz

# remove uninterested entities e.g. "RT" or "Golden Globes"
def preprocess(entity_dict):
    entity_dict = entity_dict.copy()
    to_remove = []
    for key in entity_dict.keys():
        if fuzz.ratio(key.lower(), 'goldenglobes') > 50 or fuzz.ratio(key.lower(), 'rt') > 50:
#             print(key)
            to_remove.append(key)

    for key in to_remove:
        del entity_dict[key]
    
    return entity_dict

# Generate Predictions

In [166]:
for key in award_num_keywords_map.keys():
    print(sorted(preprocess(find_award_winner(key, verbose=False)).items(), key=lambda pair: pair[1], reverse=True)[:100])
    print()

Predicting for: best actress in a motion picture comedy or musical
num of matches: 8
[('Jennifer Lawrence', 8), ('ABC', 7), ('Silver Lining Playbook', 7), ('KNXDavid', 1), ('KNX1070', 1)]

Predicting for: best television series actor drama
num of matches: 0
[]

Predicting for: best actress in a motion picture musical or comedy
num of matches: 8
[('Jennifer Lawrence', 8), ('ABC', 7), ('Silver Lining Playbook', 7), ('KNXDavid', 1), ('KNX1070', 1)]

Predicting for: best supporting actor in a motion picture
num of matches: 0
[]

Predicting for: best supporting actor actress
num of matches: 1
[('Christoph Waltz', 1), ('Django', 1), ('Maggie Smith', 1), ('Downton Abbey', 1)]

Predicting for: best supporting actress in a motion picture
num of matches: 2
[('Anne Hathaway', 2), ('Hooray', 1), ('Winner', 1), ('KNXDavid', 1), ('Les Miserables', 1), ('KNX1070', 1)]

Predicting for: best actor in a motion picture comedy or musical
num of matches: 65
[('Hugh Jackman', 65), ('Les Miserables', 63), ('